<a href="https://colab.research.google.com/github/Gratisfo/project_ml/blob/main/%D0%A2%D0%B5%D0%BC%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%B8%D1%81%D0%B5%D0%BC_%D0%9A%D0%BB%D0%B8%D0%BD%D1%82%D0%BE%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задача о тематическом моделировании




### Постановка задачи
Загрузите [коллекцию писем Х. Клинтон](https://www.kaggle.com/kaggle/hillary-clinton-emails/) с kaggle. Для скачивания может потребоваться регистрация.

Методом LDA выделите несколько тем в переписке Х. Клинтон, дайте им словесное описание. Используйте библиотеку LdaModel из gensim.


In [ ]:
! pip install pyLDAvis

In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from scipy.sparse import csc
from gensim import matutils

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# spacy для лемматизции
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings

In [ ]:
import pandas as pd
df = pd.read_csv('Emails.csv')

In [ ]:
df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,MetadataDocumentClass,ExtractedSubject,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,HRC_Email_296,FW: Wow,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,HRC_Email_296,Re: Chris Stevens,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,HRC_Email_296,FVV: Cairo Condemnation - Final,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


In [ ]:
# препроцессинг для текстов 

data = df.dropna(subset=['ExtractedBodyText'])['ExtractedBodyText'].tolist()

# удаляем емейлы
data = [re.sub(r"\S*@\S*\s?", "", text) for text in data]
# удаляем символы новой строки
data = [re.sub(r"\s+", " ", text) for text in data]
# удаляем некоторые чиселки
data = [re.sub(r"\d.*?\s", "", text) for text in data]

In [ ]:
# разделяем тексты на слова
def text_to_words(data):
    for text in data:
        yield(gensim.utils.simple_preprocess(str(text), deacc=True))  # deacc=True удаляем пунктуацию
data_words = list(text_to_words(data))

In [ ]:
# удаляем стоп слова
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_nostops = remove_stopwords(data_words)

In [ ]:
# лемматизируем тексты, сохраняем только noun, adj, vb, adv
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
data_lemmatized = [text for text in data_lemmatized if len(text) > 0]
data_texts = [' '.join(text) for text in data_lemmatized if len(text) > 0]

In [ ]:
# Считаем векторы CountVectorizer
vec = CountVectorizer(min_df=10)
# Term Document Frequency
X_CV = vec.fit_transform(data_texts)
# Создаем корпус
corpus_CV = matutils.Sparse2Corpus(X_CV.T)
# Создаем словарь
vect_dict_CV = vec.get_feature_names()
dictionary_CV = gensim.corpora.Dictionary.from_corpus(corpus_CV, id2word=dict([(i, s) for i, s in enumerate(vect_dict_CV)]))

In [ ]:
# Функция для фита модели lda
from gensim.models.ldamodel import LdaModel
from gensim import matutils

def fit_lda(corpus, dictionary, num_topics=5, passes=20):
    """ Fit LDA from a scipy CSR matrix (X). """
    print('fitting lda...')
    return LdaModel(corpus, num_topics=num_topics,
                    passes=passes,
                    id2word=dictionary)

In [ ]:
# Fit LDA CountVectorizer
lda_CV = fit_lda(corpus_CV, dictionary_CV)

In [ ]:
# Cписки ключевых слов по CountVectorizer
lda_CV.print_topics()

[(0,
  '0.069*"call" + 0.024*"send" + 0.023*"state" + 0.020*"today" + 0.018*"information" + 0.018*"case" + 0.015*"print" + 0.015*"house" + 0.014*"sensitive" + 0.014*"tomorrow"'),
 (1,
  '0.037*"office" + 0.035*"pm" + 0.032*"meeting" + 0.032*"private" + 0.031*"time" + 0.029*"arrive" + 0.025*"residence" + 0.025*"call" + 0.017*"cdm" + 0.016*"meet"'),
 (2,
  '0.019*"go" + 0.018*"see" + 0.018*"know" + 0.018*"get" + 0.017*"think" + 0.017*"say" + 0.016*"would" + 0.015*"work" + 0.015*"want" + 0.015*"good"'),
 (3,
  '0.009*"government" + 0.008*"woman" + 0.008*"state" + 0.008*"work" + 0.007*"support" + 0.007*"country" + 0.007*"need" + 0.007*"people" + 0.007*"security" + 0.007*"would"'),
 (4,
  '0.015*"say" + 0.008*"would" + 0.006*"year" + 0.006*"right" + 0.005*"political" + 0.005*"government" + 0.005*"time" + 0.005*"american" + 0.005*"party" + 0.004*"even"')]

In [ ]:
# визуализация CountVectorizer
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_CV, corpus_CV, dictionary_CV)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.148136  0.057602       1        1  44.862540
3      0.166508  0.116097       2        1  24.007773
2      0.062350 -0.091624       3        1  19.504796
0     -0.098772 -0.250692       4        1   6.859288
1     -0.278222  0.168616       5        1   4.765603, topic_info=          Term         Freq        Total Category  logprob  loglift
291       call  1495.000000  1495.000000  Default  30.0000  30.0000
2231      time  1095.000000  1095.000000  Default  29.0000  29.0000
1476    office   422.000000   422.000000  Default  28.0000  28.0000
2091     state   883.000000   883.000000  Default  27.0000  27.0000
1346   meeting   412.000000   412.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
1937  schedule    68.430701   170.901460   Topic5  -4.8630   2.1285
2061   special    68.609528   185.054875   Topic5  -4.8604   2.0515
1333       may   103.065561   789.560196   Topic5  -4.4535   1.0076
2031     small    70.512165   209.639088   Topic5  -4.8331   1.9541
1424      need    86.814761   713.066031   Topic5  -4.6251   0.9379

[366 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
14        1  0.930777  accord
14        2  0.028205  accord
14        3  0.037607  accord
20        1  0.987419  accuse
31        3  0.960832  actual
...     ...       ...     ...
2436      4  0.040000   would
2436      5  0.001905   would
2443      1  0.641614    year
2443      2  0.181950    year
2443      3  0.175965    year

[734 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 1, 2])

In [ ]:
# Считаем векторы TfidfVectorizer
vectorizer = TfidfVectorizer()
# Создаем матрицу и корпус
X_TFIDF = vectorizer.fit_transform(data_texts)
corpus_tfidf = gensim.matutils.Sparse2Corpus(csc.csc_matrix(X_TFIDF))
# Создаем словарь
vect_dict_tfidf = vectorizer.get_feature_names()
dictionary_tfidf = gensim.corpora.Dictionary.from_corpus(
    corpus_tfidf,
    id2word=dict([(i, s) for i, s in enumerate(vect_dict_tfidf)]))

In [ ]:
# Fit LDA TfidfVectorizer
lda_TFIDF = fit_lda(corpus_tfidf, dictionary_tfidf)

In [ ]:
# Cписки ключевых слов по TfidfVectorizer
lda_TFIDF.print_topics()

[(0,
  '0.002*"bona" + 0.002*"dore" + 0.002*"expectation" + 0.002*"expectancy" + 0.002*"celtic" + 0.002*"embodie" + 0.002*"bold" + 0.002*"casualty" + 0.002*"essentially" + 0.002*"drug"'),
 (1,
  '0.002*"biased" + 0.002*"golitic" + 0.002*"crush" + 0.002*"gaming" + 0.002*"dip" + 0.002*"darling" + 0.002*"fwd" + 0.002*"beer" + 0.002*"eater" + 0.002*"eye"'),
 (2,
  '0.003*"finger" + 0.003*"entertain" + 0.003*"dresser" + 0.003*"fend" + 0.003*"experiment" + 0.003*"flung" + 0.003*"efe" + 0.003*"compile" + 0.003*"devotedly" + 0.003*"enroll"'),
 (3,
  '0.002*"expression" + 0.002*"discourage" + 0.002*"economic" + 0.002*"carol" + 0.002*"format" + 0.002*"culminate" + 0.002*"deviation" + 0.002*"analyse" + 0.002*"desert" + 0.002*"express"'),
 (4,
  '0.002*"grove" + 0.002*"aspstorie" + 0.002*"dayton" + 0.002*"fixation" + 0.002*"contrib" + 0.002*"adventure" + 0.002*"bwere" + 0.002*"disentangling" + 0.002*"addressing" + 0.002*"caliber"'),
 (5,
  '0.002*"currency" + 0.002*"cave" + 0.002*"disabilitie" + 0

In [ ]:
# визуализация TfidfVectorizer
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_TFIDF, corpus_tfidf, dictionary_tfidf)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.246151 -0.033717       1        1  44.411858
3     -0.123186 -0.128999       2        1  27.272594
2     -0.050652  0.087707       3        1  12.155419
4     -0.045794  0.060167       4        1  10.724187
1     -0.026519  0.014841       5        1   5.435942, topic_info=            Term       Freq      Total Category  logprob  loglift
4639      finger  17.000000  17.000000  Default  30.0000  30.0000
4058   entertain  17.000000  17.000000  Default  29.0000  29.0000
3653     dresser  16.000000  16.000000  Default  28.0000  28.0000
4563        fend  15.000000  15.000000  Default  27.0000  27.0000
4334  experiment  15.000000  15.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
5463      hasten   0.979051   1.547572   Topic5  -6.8653   2.4543
4812     foresee   1.051921   1.858322   Topic5  -6.7935   2.3431
3148     deplore   1.006403   2.082887   Topic5  -6.8377   2.1848
3016      defame   0.968067   1.817211   Topic5  -6.8765   2.2824
2926      dayton   0.937370   2.777632   Topic5  -6.9088   1.8259

[208 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2         4  0.729887  abandonment
23        4  0.953827     abortion
35        4  0.667538      abstain
68        4  0.953827      acclaim
98        1  0.992426   accusation
...     ...       ...          ...
5289      3  0.804473         grey
5315      2  0.831104    grotesque
5322      2  0.774951        grove
5415      5  0.749009    handpicke
5463      5  0.646173       hasten

[183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 5, 2])

In [ ]:
# Compute Perplexity
perplexity_cv = lda_CV.log_perplexity(corpus_CV)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_CV, texts=data_lemmatized, dictionary=dictionary_CV, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [ ]:
print('\nPerplexity CountVectorizer: ', perplexity_cv)
print('\nCoherence Score CountVectorizer: ', coherence_lda)


Perplexity CountVectorizer:  -7.009400718785396

Coherence Score CountVectorizer:  0.3937092413248971


In [ ]:
# Compute Perplexity
perplexity_tfidf = lda_TFIDF.log_perplexity(corpus_tfidf)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_TFIDF, texts=data_lemmatized, dictionary=dictionary_tfidf, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()


In [ ]:
print('\nPerplexity TfidfVectorizer: ', perplexity_tfidf) 
print('\nCoherence Score TfidfVectorizer: ', coherence_lda)


Perplexity TfidfVectorizer:  -9.939079520511322

Coherence Score TfidfVectorizer:  0.7521333380846583
